# Import liberies

In [ ]:
import warnings
import sklearn
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
import keras 
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, GlobalAvgPool2D, Conv2D
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import confusion_matrix, accuracy_score
from PIL import Image, ImageOps, ImageEnhance

import os
import re
import pandas as pd
import numpy as np

# Data processing

## Prepare

Convert Images to train test dataset

In [ ]:

# def convert_images_to_dataset():
#     ls = []
#     image_data=[]
#     for root, _, files in os.walk("/kaggle/input/rice-leaf-diseases"):
#         if len(root) > len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases"):
#             label = root[len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases")+1:]
#         for filename in files:
#             image_path = os.path.join(root, filename)
#             image_data.append(image_path)
#             if label == "Bacterial leaf blight":
#                 ls.append(0)
#             elif label == "Brown spot":
#                 ls.append(1)
#             elif label == "Leaf smut":
#                 ls.append(2)
#     return image_data, ls
# image_data, ls = convert_images_to_dataset()
# df = pd.DataFrame()
# df["image_path"] = image_data
# df["label"] = ls
# df = df.sample(frac=1, random_state=42)  # Set a random_state for reproducibility
# df.reset_index(drop=True, inplace=True)
# df.head()

In [ ]:
def info_folder(folder):
    for i in os.listdir(folder):
        print(i,len(os.listdir(folder+"/"+i)))

In [ ]:
info_folder("/kaggle/input/rice-disease-dataset/dataset/train")

## Transform

In [ ]:
# ohencoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
# ohlabel = pd.DataFrame(ohencoder.fit_transform(df[['label']]),dtype = 'float64',columns = ['label0','label1','label2'])
# label_X = df.copy()
# df = pd.concat([df,ohlabel],axis = 1)
# new_df = df.drop(['label'],axis = 1)

## Split

With dataframe

In [ ]:
def resized_image(data):
    flat = []
    for i in data:
        image = Image.open(i)
        width, height = image.size
        new_width = 224
        new_height = 224
        resized_image = image.resize((new_width, new_height))
        img_array = np.array(resized_image) / 255.0
        flat.append(img_array)
    processed_images = np.array(flat).reshape(-1, 224, 224, 3)
    return processed_images

In [ ]:
# train,test = train_test_split(new_df,test_size=0.2,random_state=32,shuffle = True)
# X_train = train['image_path'].values
# y_train = train.drop(['image_path'],axis=1).values
# X_test = test['image_path'].values
# y_test = test.drop(['image_path'],axis=1).values
# dx_train = resized_image(X_train)
# dx_test = resized_image(X_test)

With directly in folder

In [ ]:
# run this on local pc since kaggle doesn't allow to change directory element
# def split_train_test(folder):   
#     try:
#         os.mkdir("train")
#         os.mkdir("test")
#     except:
#         pass
#     for i in os.listdir(folder):
#         os.mkdir("train/"+i)
#         os.mkdir("test/"+i)
#         for j in os.listdir(folder+"/"+i)[:400]:
#             os.rename(folder+"/"+i+"/"+j, "train/"+i+"/"+j)
#         for j in os.listdir(folder+"/"+i)[:100]:
#             os.rename(folder+"/"+i+"/"+j, "test/"+i+"/"+j)

In [ ]:
# split_train_test("/kaggle/input/rice-leaf-images/rice_images")

In [ ]:
def img_data(dir_path,
             target_size,
             batch_size,
             class_lst,
             preprocessing,):
    if preprocessing:
        gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
    else:
        gen_object = ImageDataGenerator()
        
    return gen_object.flow_from_directory(dir_path,
                                      target_size=target_size,
                                      batch_size=batch_size, 
                                      classes=class_lst,
                                      shuffle=True)

In [ ]:
class_lst = os.listdir("/kaggle/input/rice-disease-dataset/dataset/train")

In [ ]:
train_data_gen =  img_data("/kaggle/input/rice-disease-dataset/dataset/train",
                          (224,224),
                           128,
                           class_lst,
                          preprocess_input)
test_data_gen =  img_data("/kaggle/input/rice-disease-dataset/dataset/test",
                          (224,224),
                           128,
                           class_lst,
                          preprocess_input)

# Model training

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(
                   include_top=False,
                   input_shape=(224,224,3),
                   pooling=None,classes=128,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
# resnet_model.add(GlobalAvgPool2D())
resnet_model.add(Flatten())
resnet_model.add(Dense(1024, activation='relu'))
resnet_model.add(Dense(4, activation='softmax'))

In [ ]:
resnet_model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck =callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True, monitor='val_loss',mode='min')

In [ ]:
history = resnet_model.fit(train_data_gen,batch_size=500,epochs=50,validation_data=test_data_gen,callbacks=[elst,save_ck])

In [ ]:
def deepmodelpipeline(imagepath,model = resnet_model,label=[-1]):
    pdict = {0:"Bacterial leaf blight",1:"Brown spot",2:"Leaf smut"}
    pred_x = resized_image([imagepath])
    prediction = model.predict(pred_x)
    pred = np.argmax(prediction[0])
    plt.imshow(plt.imread(imagepath))
    if (label[0]!=-1):
        plt.title("prediction : {0} % {1:.2f} \ntrue        : {2}".format(pdict[pred],prediction[0,pred]*100,pdict[np.argmax(label)]))
    else:
        plt.title("prediction : {0}, % {1:.2f}".format(pdict[pred],prediction[0,pred]*100))

In [ ]:
image_path = "/kaggle/input/rice-leaf-diseases/rice_leaf_diseases/Brown spot/DSC_0100.jpg"
deepmodelpipeline(image_path)

In [ ]:
import pickle
with open("model.pkl", 'wb') as model_file:
    pickle.dump(resnet_model, model_file)


### Creating requirements.txt

In [ ]:
print("scikit-learn==", sklearn.__version__)
print("TensorFlow==", tf.__version__)
print("Keras version==", keras.__version__)
print("h5py version==", h5py.__version__)
print("Pillow==", Image.__version__)
print("pandas==", pd.__version__)
print("NumPy==", np.__version__)